In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [3]:
REBUILD_DATA = False

In [4]:
class DogsVsCats():
    IMG_SIZE = 50
    CATS = "kagglecatsanddogs_3367a/PetImages/Cat/"
    DOGS = "kagglecatsanddogs_3367a/PetImages/Dog/"
    LABELS = {CATS: 0, DOGS: 1}
    
    training_data = [] 
    cat_count = 0
    dog_count = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            
            for f in tqdm(os.listdir(label)): # TQDM is used to show the progress status
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                    if label == self.CATS:
                        self.cat_count += 1
                    elif label == self.DOGS:
                        self.dog_count += 1
                except Exception as e:
                    print(str(e))
                    continue
        np.random.shuffle(self.training_data)  # It is in place. So no need to reassign
        np.save("training_data.npy", self.training_data)
        print("Cats:", self.cat_count)
        print("Dogs:", self.dog_count)

In [5]:
if REBUILD_DATA:
    dogsvcats = DogsVsCats()
    dogsvcats.make_training_data()

In [6]:
training_data = np.load("training_data.npy", allow_pickle = True)

In [7]:
print((len(training_data)))

24946


In [8]:
print(training_data[1])

[array([[ 85,  91, 121, ..., 100,  87, 114],
       [ 68, 118, 115, ..., 119,  83, 111],
       [ 80,  93, 178, ..., 142, 113,  74],
       ...,
       [ 24,  24,  28, ...,  24,  21,  20],
       [ 21,  24,  26, ...,  21,  20,  18],
       [ 20,  24,  24, ...,  20,  20,  16]], dtype=uint8)
 array([1., 0.])]


In [9]:
import matplotlib.pyplot as plt

plt.imshow(training_data[1][0], cmap = "gray")
plt.show()

<Figure size 640x480 with 1 Axes>

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) # Input, Output, Kernel
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        x = torch.randn(50,50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2] # Alternative is np.prd(x.shape)
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.softmax(x, dim =1)

In [11]:
net = Net()

torch.Size([128, 2, 2])


In [12]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr = 0.001)

loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0

y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)

print(val_size)

2494


In [13]:
val_size

2494

In [14]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

22452
2494


In [ ]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        #print(i, i+BATCH_SIZE)
        batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
        batch_y = train_y[i:i+BATCH_SIZE]
        
        net.zero_grad() # or optimizer.zero_grad()
        
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
print(loss)

  0%|          | 0/225 [00:00<?, ?it/s]